# Load and Embed PDF artciles
[Following](https://www.linkedin.com/pulse/build-lightning-fast-rag-chatbot-powered-groqs-lpu-ollama-multani-ssloc)

In [1]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import TextLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community import embeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# from google.colab import userdata
import os
import time
import textwrap
import gradio as gr

__groq_api_key__ = os.environ.get("GROQ_API_KEY")

## Loading documents

In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

data_dir = "/home/nuwan/workspace/penman/wrangler/data/article/stock/cap"
loader = PyPDFDirectoryLoader(data_dir)

the_text = loader.load()    

## Splitting Text into Chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks = text_splitter.split_documents(the_text)

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

vectorstore = Chroma.from_documents(

    documents=chunks,

    collection_name="ollama_embeds",

    embedding=OllamaEmbeddings(model='nomic-embed-text'),

)

## Setting Up Groq's LPU for Inference

In [5]:
from langchain_groq import ChatGroq

# from google.colab import userdata

# groq_api_key = userdata.get('groq_api_key')

llm = ChatGroq(

            groq_api_key=__groq_api_key__,

            model_name='mixtral-8x7b-32768'

    )

## Building the RAG Chain

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = vectorstore.as_retriever()

rag_template = """..."""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

## Testing the RAG Architecture

In [12]:
response = rag_chain.invoke("What is this document about")
print(textwrap.fill(response, width=80))

I'm here to help! To provide the best assistance, could you please provide me
with a little more information about what you're looking for? For example, are
you interested in learning a new programming language, do you need help with a
specific coding problem, or are you looking for resources to learn about a
particular technology? I'm here to answer any questions you have to the best of
my ability.


## Launching the Gradio Interface

In [13]:
def process_question(user_question):
    # Processing and response time measurement here
    iface = gr.Interface(fn=process_question,
                         inputs=gr.Textbox(lines=2, placeholder="Type your question here..."),
                         outputs=gr.Textbox(),
                         title="GROQ CHAT",
                         description="Ask any question about your document, and get an answer along with the response time.")
    iface.launch()

In [14]:
user_question = "what is cap?"
process_question(user_question)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 2014, in process_api
    result = await self.call_function(
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1567, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_asy

Created dataset file at: .gradio/flagged/dataset1.csv


Traceback (most recent call last):
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 2014, in process_api
    result = await self.call_function(
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1567, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_asy